In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import logging
import unittest
import colorama

from communication.message.MessagePerformative import MessagePerformative
from communication.preferences.CriterionName import CriterionName
from communication.preferences.CriterionValue import CriterionValue
from communication.preferences.Item import Item
from communication.preferences.Preferences import Preferences
from communication.preferences.Value import Value
from pw_argumentation import ArgumentModel, format_argument

colorama.init()  # INFO: used to print colored text on Windows

An example conversation between two agents

In [9]:
argument_model = ArgumentModel()
argument_model.run_model()

results, history = argument_model.get_final_result()
if results:
    print("Winning agent:", results["winning_agent"])
    print("Winning item:", results["winning_item"])
    print("Winning argument:", format_argument(results["winning_argument"]))

history

Winning agent: Bob
Winning item: Engine 1
Winning argument: Engine 1, PRODUCTION_COST==GOOD


,sender,receiver,performative,item,decision,main_criterion,value,secondary_criterion
0,Bob,Alice,PROPOSE,Engine 5,None,None,None,None
1,Alice,Bob,ASK_WHY,Engine 5,None,None,None,None
2,Bob,Alice,ARGUE,Engine 5,pro,CriterionName.DURABILITY,Value.VERY_GOOD,None
3,Alice,Bob,ARGUE,Engine 5,con,CriterionName.CONSUMPTION,Value.BAD,CriterionName.DURABILITY
4,Bob,Alice,REJECT,Engine 5,None,None,None,None
5,Alice,Bob,PROPOSE,Engine 3,None,None,None,None
6,Bob,Alice,ASK_WHY,Engine 3,None,None,None,None
7,Alice,Bob,ARGUE,Engine 3,pro,CriterionName.CONSUMPTION,Value.VERY_GOOD,None
8,Bob,Alice,ARGUE,Engine 3,con,CriterionName.DURABILITY,Value.VERY_BAD,CriterionName.CONSUMPTION
9,Alice,Bob,REJECT,Engine 3,None,None,None,None


## Set up agents

,sender,receiver,performative,item,decision,main_criterion,value,secondary_criterion
0,Bob,Alice,PROPOSE,Engine 1,None,None,None,None
1,Alice,Bob,ASK_WHY,Engine 1,None,None,None,None
2,Bob,Alice,PROPOSE,Engine 2,None,None,None,None
3,Alice,Bob,ASK_WHY,Engine 2,None,None,None,None
4,Bob,Alice,PROPOSE,Engine 1,None,None,None,None
5,Alice,Bob,ASK_WHY,Engine 1,None,None,None,None
6,Bob,Alice,PROPOSE,Engine 2,None,None,None,None
7,Alice,Bob,ASK_WHY,Engine 2,None,None,None,None
8,Bob,Alice,PROPOSE,Engine 1,None,None,None,None
9,Alice,Bob,ASK_WHY,Engine 1,None,None,None,None


## Start negotiations

## Statistics